Import das Bibliotecas

In [1]:
import requests
import pandas as pd

Importacao e Preparacao Inicial da Base

In [2]:
control = {
    "euro": {
        "it": {
            "interval": (1583539200000, 1593043200000), # 07/03/2020 00:00 - 25/06/2020 00:00 (GMT)
            "population": 60453718
        },
        "es": {
            "interval": (1584144000000, 1592611200000), # 14/03/2020 00:00 - 20/06/2020 00:00 (GMT)
            "population": 46756434
        },
        "fr": {
            "interval": (1584403200000, 1592784000000),  # 17/03/2020 00:00 - 22/06/2020 00:00 (GMT)
            "population": 65286749
        }
    },
    #"searo": {
    #    "th": {
    #        "interval": (1585008000000, 1593561600000), # 24/03/2020 00:00 - 01/07/2020 00:00 (GMT)
    #        "population":  69799978
    #    }
    #}
}

In [3]:
data = {}
headers = []

for i, continent in enumerate(control):
    
    for j, country in enumerate(control[continent]):
        
        print(continent+"_"+country)
        url = "https://covid19.who.int/page-data/region/{}/country/{}/page-data.json".format(continent, country)
        print(url)
        
        res = requests.get(url, timeout=30).json()
        
        if i == 0 and j == 0:
            
            headers = res["result"]["pageContext"]["byDay"]["metrics"]
            
        data[continent+"_"+country] = res["result"]["pageContext"]["byDay"]["rows"]

euro_es
https://covid19.who.int/page-data/region/euro/country/es/page-data.json
euro_it
https://covid19.who.int/page-data/region/euro/country/it/page-data.json
euro_fr
https://covid19.who.int/page-data/region/euro/country/fr/page-data.json


In [4]:
print([entry["name"] for entry in headers])
print(data.keys())
print([len(data[key]) for key in data])
print(data["euro_it"][0])

['Deaths', 'Cumulative Deaths', 'Deaths Last 7 Days', 'Deaths Last 7 Days Change', 'Deaths Per Million', 'Confirmed', 'Cumulative Confirmed', 'Cases Last 7 Days', 'Cases Last 7 Days Change', 'Cases Per Million']
dict_keys(['euro_es', 'euro_fr', 'euro_it'])
[183, 192, 187]
[1580256000000, 'EURO', 0, 0, 0, 0, 0, 6, 6, 6, 0, 0]


In [5]:
columns = ["Date","Continent"]+[entry["name"] for entry in headers]

dfs = None

for key in data:
    
    continent,country = key.split("_")
    
    df = pd.DataFrame(data[key], columns=columns)
    df["Country"] = country.upper()
    df["Population"] = control[continent][country]["population"]
    
    if dfs is None:
        
        dfs = df
        
    else:
        
        dfs = pd.concat([dfs, df], ignore_index = True)

Análise e Preparação dos Dados

In [6]:
dfs

,Date,Continent,Deaths,Cumulative Deaths,Deaths Last 7 Days,Deaths Last 7 Days Change,Deaths Per Million,Confirmed,Cumulative Confirmed,Cases Last 7 Days,Cases Last 7 Days Change,Cases Per Million,Country,Population
0,1580601600000,EURO,0,0,0,0,0,5,5,5,0,0,ES,46756434
1,1580688000000,EURO,0,0,0,0,0,0,5,5,0,0,ES,46756434
2,1580774400000,EURO,0,0,0,0,0,0,5,5,0,0,ES,46756434
3,1580860800000,EURO,0,0,0,0,0,0,5,5,0,0,ES,46756434
4,1580947200000,EURO,0,0,0,0,0,0,5,5,0,0,ES,46756434
5,1581033600000,EURO,0,0,0,0,0,0,5,5,0,0,ES,46756434
6,1581120000000,EURO,0,0,0,0,0,0,5,5,0,0,ES,46756434
7,1581206400000,EURO,0,0,0,0,0,0,5,0,-100,0,ES,46756434
8,1581292800000,EURO,0,0,0,0,0,1,6,1,-80,0,ES,46756434
9,1581379200000,EURO,0,0,0,0,0,0,6,1,-80,0,ES,46756434


In [7]:
dfs.to_csv('..\\..\\Normalizado\\Indicadores\\WHO_Reference.csv'.format(data), sep=';', index=False)

Filtragem para Base de Modelo

In [8]:
df = dfs[['Date', 'Deaths Last 7 Days', 'Cases Last 7 Days', 'Deaths Last 7 Days Change', 'Cases Last 7 Days Change', 'Country', 'Population']]

In [9]:
df

,Date,Deaths Last 7 Days,Cases Last 7 Days,Deaths Last 7 Days Change,Cases Last 7 Days Change,Country,Population
0,1580601600000,0,5,0,0,ES,46756434
1,1580688000000,0,5,0,0,ES,46756434
2,1580774400000,0,5,0,0,ES,46756434
3,1580860800000,0,5,0,0,ES,46756434
4,1580947200000,0,5,0,0,ES,46756434
5,1581033600000,0,5,0,0,ES,46756434
6,1581120000000,0,5,0,0,ES,46756434
7,1581206400000,0,0,0,-100,ES,46756434
8,1581292800000,0,1,0,-80,ES,46756434
9,1581379200000,0,1,0,-80,ES,46756434


In [10]:
for continent in control:
    
    for country in control[continent]:

        df = df.loc[((df["Country"] == country.upper()) & 
                    (df["Date"] >= control[continent][country]["interval"][0]) & 
                    (df["Date"] <= control[continent][country]["interval"][1])) |
                    (df["Country"] != country.upper())]

In [11]:
df

,Date,Deaths Last 7 Days,Cases Last 7 Days,Deaths Last 7 Days Change,Cases Last 7 Days Change,Country,Population
41,1584144000000,328,3857,3180,1027,ES,46756434
42,1584230400000,443,5379,1745,1534,ES,46756434
43,1584316800000,603,7379,1407,2295,ES,46756434
44,1584403200000,763,8167,1073,797,ES,46756434
45,1584489600000,962,9556,881,549,ES,46756434
46,1584576000000,1214,11572,698,496,ES,46756434
47,1584662400000,1554,14178,584,424,ES,46756434
48,1584748800000,1944,15745,492,308,ES,46756434
49,1584835200000,2350,19169,430,256,ES,46756434
50,1584921600000,2833,20815,369,182,ES,46756434


In [12]:
quarantine_ends = df.groupby(["Country"]).agg({'Date': ['max']}).to_dict()[('Date', 'max')]
quarantine_ends

{'ES': 1592611200000, 'FR': 1592784000000, 'IT': 1593043200000}

In [13]:
def reference(row):
    
    return quarantine_ends[row["Country"]]

In [14]:
df["Reference"] = df.apply(lambda row: reference(row), axis=1)

In [15]:
df

,Date,Deaths Last 7 Days,Cases Last 7 Days,Deaths Last 7 Days Change,Cases Last 7 Days Change,Country,Population,Reference
41,1584144000000,328,3857,3180,1027,ES,46756434,1592611200000
42,1584230400000,443,5379,1745,1534,ES,46756434,1592611200000
43,1584316800000,603,7379,1407,2295,ES,46756434,1592611200000
44,1584403200000,763,8167,1073,797,ES,46756434,1592611200000
45,1584489600000,962,9556,881,549,ES,46756434,1592611200000
46,1584576000000,1214,11572,698,496,ES,46756434,1592611200000
47,1584662400000,1554,14178,584,424,ES,46756434,1592611200000
48,1584748800000,1944,15745,492,308,ES,46756434,1592611200000
49,1584835200000,2350,19169,430,256,ES,46756434,1592611200000
50,1584921600000,2833,20815,369,182,ES,46756434,1592611200000


In [16]:
df["Label"] = (df["Reference"] - df["Date"])/1000/3600/24
df["deaths7perc"] = (df["Deaths Last 7 Days"]/df["Population"]) * 100
df["cases7perc"] = (df["Cases Last 7 Days"]/df["Population"]) * 100

In [17]:
df

,Date,Deaths Last 7 Days,Cases Last 7 Days,Deaths Last 7 Days Change,Cases Last 7 Days Change,Country,Population,Reference,Label,deaths7perc,cases7perc
41,1584144000000,328,3857,3180,1027,ES,46756434,1592611200000,98.0,0.000702,0.008249
42,1584230400000,443,5379,1745,1534,ES,46756434,1592611200000,97.0,0.000947,0.011504
43,1584316800000,603,7379,1407,2295,ES,46756434,1592611200000,96.0,0.001290,0.015782
44,1584403200000,763,8167,1073,797,ES,46756434,1592611200000,95.0,0.001632,0.017467
45,1584489600000,962,9556,881,549,ES,46756434,1592611200000,94.0,0.002057,0.020438
46,1584576000000,1214,11572,698,496,ES,46756434,1592611200000,93.0,0.002596,0.024750
47,1584662400000,1554,14178,584,424,ES,46756434,1592611200000,92.0,0.003324,0.030323
48,1584748800000,1944,15745,492,308,ES,46756434,1592611200000,91.0,0.004158,0.033675
49,1584835200000,2350,19169,430,256,ES,46756434,1592611200000,90.0,0.005026,0.040998
50,1584921600000,2833,20815,369,182,ES,46756434,1592611200000,89.0,0.006059,0.044518


In [18]:
df = df[['Deaths Last 7 Days', 'Cases Last 7 Days', 'Deaths Last 7 Days Change', 'Cases Last 7 Days Change', 'deaths7perc', 'cases7perc', 'Country', 'Label']]

In [19]:
df

,Deaths Last 7 Days,Cases Last 7 Days,Deaths Last 7 Days Change,Cases Last 7 Days Change,deaths7perc,cases7perc,Country,Label
41,328,3857,3180,1027,0.000702,0.008249,ES,98.0
42,443,5379,1745,1534,0.000947,0.011504,ES,97.0
43,603,7379,1407,2295,0.001290,0.015782,ES,96.0
44,763,8167,1073,797,0.001632,0.017467,ES,95.0
45,962,9556,881,549,0.002057,0.020438,ES,94.0
46,1214,11572,698,496,0.002596,0.024750,ES,93.0
47,1554,14178,584,424,0.003324,0.030323,ES,92.0
48,1944,15745,492,308,0.004158,0.033675,ES,91.0
49,2350,19169,430,256,0.005026,0.040998,ES,90.0
50,2833,20815,369,182,0.006059,0.044518,ES,89.0


In [20]:
def absoluteToBoolean(row, key):
    
    if row[key] > 0:
        
        return 1
        
    else:
        
        return 0

In [21]:
df.loc[df['Deaths Last 7 Days Change'] > 0, 'Deaths Last 7 Days Change'] = 1
df.loc[df['Deaths Last 7 Days Change'] <= 0, 'Deaths Last 7 Days Change'] = 0

df.loc[df['Cases Last 7 Days Change'] > 0, 'Cases Last 7 Days Change'] = 1
df.loc[df['Cases Last 7 Days Change'] <= 0, 'Cases Last 7 Days Change'] = 0

C:\Users\DELL\.conda\envs\hackatonV2\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [22]:
df

,Deaths Last 7 Days,Cases Last 7 Days,Deaths Last 7 Days Change,Cases Last 7 Days Change,deaths7perc,cases7perc,Country,Label
41,328,3857,1,1,0.000702,0.008249,ES,98.0
42,443,5379,1,1,0.000947,0.011504,ES,97.0
43,603,7379,1,1,0.001290,0.015782,ES,96.0
44,763,8167,1,1,0.001632,0.017467,ES,95.0
45,962,9556,1,1,0.002057,0.020438,ES,94.0
46,1214,11572,1,1,0.002596,0.024750,ES,93.0
47,1554,14178,1,1,0.003324,0.030323,ES,92.0
48,1944,15745,1,1,0.004158,0.033675,ES,91.0
49,2350,19169,1,1,0.005026,0.040998,ES,90.0
50,2833,20815,1,1,0.006059,0.044518,ES,89.0


In [23]:
df = df.rename(columns={"Deaths Last 7 Days": "Deaths7",
                        "Cases Last 7 Days": "Cases7",
                        "Deaths Last 7 Days Change": "Deaths7Trend",
                        "Cases Last 7 Days Change": "Cases7Trend"})

In [24]:
df

,Deaths7,Cases7,Deaths7Trend,Cases7Trend,deaths7perc,cases7perc,Country,Label
41,328,3857,1,1,0.000702,0.008249,ES,98.0
42,443,5379,1,1,0.000947,0.011504,ES,97.0
43,603,7379,1,1,0.001290,0.015782,ES,96.0
44,763,8167,1,1,0.001632,0.017467,ES,95.0
45,962,9556,1,1,0.002057,0.020438,ES,94.0
46,1214,11572,1,1,0.002596,0.024750,ES,93.0
47,1554,14178,1,1,0.003324,0.030323,ES,92.0
48,1944,15745,1,1,0.004158,0.033675,ES,91.0
49,2350,19169,1,1,0.005026,0.040998,ES,90.0
50,2833,20815,1,1,0.006059,0.044518,ES,89.0


In [25]:
df.to_csv('..\\..\\Normalizado\\Indicadores\\WHO_ReferenceV2.csv'.format(data), sep=';', index=False)